# Lab 3.8.1: Classifiers—hsbdemo Dataset
Step 1: Create a classifier using logistic regression as well as kNN from the reading, writing, mathematics, and science scores of the high-school students.<p>
Step 2: Evaluate the classifiers' accuracies in predicting which academic program the student will be joining.<p>
Step 3: Compare these two classifiers (at least their accuracies).

In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

import statsmodels.api as sm
import statsmodels.formula.api as smf

In [38]:
df = pd.read_csv('LAB 3.8.1-hsbdemo.csv')
print(df.columns)
# df.columns = ['']
display(df)

Index(['Unnamed: 0', 'id', 'female', 'ses', 'schtyp', 'prog', 'read', 'write',
       'math', 'science', 'socst', 'honors', 'awards', 'cid'],
      dtype='object')


,Unnamed: 0,id,female,ses,schtyp,prog,read,write,math,science,socst,honors,awards,cid
0,1,45,female,low,public,vocation,34,35,41,29,26,not enrolled,0,1
1,2,108,male,middle,public,general,34,33,41,36,36,not enrolled,0,1
2,3,15,male,high,public,vocation,39,39,44,26,42,not enrolled,0,1
3,4,67,male,low,public,vocation,37,37,42,33,32,not enrolled,0,1
4,5,153,male,middle,public,vocation,39,31,40,39,51,not enrolled,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,196,100,female,high,public,academic,63,65,71,69,71,enrolled,5,20
196,197,143,male,middle,public,vocation,63,63,75,72,66,enrolled,4,20
197,198,68,male,middle,public,academic,73,67,71,63,66,enrolled,7,20
198,199,57,female,middle,public,academic,71,65,72,66,56,enrolled,5,20


In [21]:
df['program'] = df['prog'].astype('category')
display(df)

,Unnamed: 0,id,female,ses,schtyp,prog,read,write,math,science,socst,honors,awards,cid,program
0,1,45,female,low,public,vocation,34,35,41,29,26,not enrolled,0,1,vocation
1,2,108,male,middle,public,general,34,33,41,36,36,not enrolled,0,1,general
2,3,15,male,high,public,vocation,39,39,44,26,42,not enrolled,0,1,vocation
3,4,67,male,low,public,vocation,37,37,42,33,32,not enrolled,0,1,vocation
4,5,153,male,middle,public,vocation,39,31,40,39,51,not enrolled,0,1,vocation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,196,100,female,high,public,academic,63,65,71,69,71,enrolled,5,20,academic
196,197,143,male,middle,public,vocation,63,63,75,72,66,enrolled,4,20,vocation
197,198,68,male,middle,public,academic,73,67,71,63,66,enrolled,7,20,academic
198,199,57,female,middle,public,academic,71,65,72,66,56,enrolled,5,20,academic


In [23]:
y.unique()

array([2, 1, 0], dtype=int8)

In [31]:
# Feature matrix (X) and target vector (y)
X = df[['read', 'write', 'math', 'science']]  # Use reading, writing, and math as features
y = df['program'].cat.codes  # 'program' is the target (categorical)

# Add a constant term (for intercept)
X = sm.add_constant(X)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the multinomial logistic regression model
model = sm.MNLogit(y_train, X_train)
result = model.fit()
print(result.summary())

# Predict on the test set
y_pred_prob = result.predict(X_test)  # Probabilities for each class

# Get the predicted class by choosing the class with the highest probability
y_pred = y_pred_prob.idxmax(axis=1)

# Calculate accuracy
accuracy_logit = accuracy_score(y_test, y_pred)

print(f"Accuracy of Logit Model predicting Program: {accuracy_logit:.2f}")

Optimization terminated successfully.
         Current function value: 0.822717
         Iterations 6
                          MNLogit Regression Results                          
Dep. Variable:                      y   No. Observations:                  160
Model:                        MNLogit   Df Residuals:                      150
Method:                           MLE   Df Model:                            8
Date:                Mon, 14 Apr 2025   Pseudo R-squ.:                  0.1827
Time:                        19:04:19   Log-Likelihood:                -131.63
converged:                       True   LL-Null:                       -161.07
Covariance Type:            nonrobust   LLR p-value:                 7.771e-10
       y=1       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.2850      1.558      2.109      0.035       0.232       6.338
read          -0.0340      0.

In [43]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize kNN classifier
knn = KNeighborsClassifier(n_neighbors=100)  # You can adjust the number of neighbors

# Train the model
knn.fit(X_train_scaled, y_train)

# Make predictions
y_pred = knn.predict(X_test_scaled)

# Calculate accuracy
accuracy_knn = accuracy_score(y_test, y_pred)
print(f"Accuracy of KNN Model predicting Program: {accuracy_knn:.2f}")

Accuracy of KNN Model predicting Program: 0.47


In [45]:
print(f"Accuracy of the Logit Model {accuracy_logit} is better than KNN {accuracy_knn}")
print(f"It's likely that there is a linear relationship between read, write, math, science and the program, so the Logit model will do better.")
print("However, neither accuracy is that good; I would not rely on either of these models.")

Accuracy of the Logit Model 0.525 is better than KNN 0.475
It's likely that there is a linear relationship between read, write, math, science and the program, so the Logit model will do better.
However, neither accuracy is that good; I would not rely on either of these models.
